In [3]:
import anndata
import scanpy as sc
import numpy as np
import pandas as pd
import post_preprocess as ppp
import re
    
"""
This loads and preprocesses the Lincs L1000 data

https://lincsproject.org/LINCS/tools/workflows/find-the-best-place-to-obtain-the-lincs-l1000-data
https://www.cell.com/cell/pdf/S0092-8674(17)31309-0.pdf

I downloaded the anndata from chemCPA, from https://dl.fbaipublicfiles.com/dlp/cpa_binaries.tar, 
"""

def remove_non_alphanumeric(input_string):
    return re.sub(r"[^a-zA-Z0-9]", "", input_string)

load_path='/home/manu/chemCPA/chemCPA/anndatas'
adata=sc.read(f'{load_path}/lincs_full.h5ad')

# It's not single cell expression data, so I will not normalize+log1p it

#sc.pp.normalize_total(adata, target_sum=1e4)
#sc.pp.log1p(adata)

In [19]:
adata.obs['cell_line']=adata.obs['cell_id']
adata.obs['treatment']=adata.obs['pert_id']
adata.obs['treatment']=[a if a!='DMSO' else 'control' for a in adata.obs['treatment']]
adata.obs["treatment"] = adata.obs["treatment"].apply(remove_non_alphanumeric)

adata.obs['treatment_dose_uM']=adata.obs['pert_dose'].astype('str')
adata.obs['treatment_dose_uM']=[a if a!='-666.0' else '0' for a in adata.obs['treatment_dose_uM']]
adata.obs['treatment_time_h']=adata.obs['pert_time']
adata.obs['treatment_type']='drug_perturbation'
adata.obs['dataset']='L1000'

adata.obs=adata.obs[['cell_line', 'treatment', 'treatment_dose_uM', 'treatment_time_h', 'treatment_type', 'dataset']]
adata.obs

,cell_line,treatment,treatment_dose_uM,treatment_time_h,treatment_type,dataset
REP.A001_A375_24H_X1_B22:A03-2,A375,control,0,24.0,drug_perturbation,L1000
REP.A001_A375_24H_X1_B22:A04-2,A375,control,0,24.0,drug_perturbation,L1000
REP.A001_A375_24H_X1_B22:A05-2,A375,control,0,24.0,drug_perturbation,L1000
REP.A001_A375_24H_X1_B22:A06-2,A375,control,0,24.0,drug_perturbation,L1000
REP.A001_A375_24H_X1_B22:A07-2,A375,BRDK25114078,10.0,24.0,drug_perturbation,L1000
...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20-1,PC3,BRDA75409952,3.32999992371,24.0,drug_perturbation,L1000
PCLB003_PC3_24H_X3_B13:P21-1,PC3,BRDA75409952,1.1100000143100002,24.0,drug_perturbation,L1000
PCLB003_PC3_24H_X3_B13:P22-1,PC3,BRDA75409952,0.3700000047680001,24.0,drug_perturbation,L1000
PCLB003_PC3_24H_X3_B13:P23-1,PC3,BRDA75409952,0.119999997318,24.0,drug_perturbation,L1000


In [31]:
# Map gene symbols to Ensemble IDs
import os
current_directory = os.getcwd()
data_path='/'+os.path.join(*current_directory.split('/')[:-1])+'/non_anndata_data'

Gene_Dict=np.load(f'{data_path}/Gene_Dict.npy', allow_pickle=True).item()
adata.var['Gene_symbol_reduced']=[remove_non_alphanumeric(a) for a in adata.var.index]
adata.var['Ensemble_ID']=[Gene_Dict[a] for a in adata.var['Gene_symbol_reduced']]
adata.var.index=adata.var['Ensemble_ID']

for c in adata.var.columns:
    del adata.var[c]

In [43]:
# We don't have an embedding for all genes, so we need to filter out these knockouts where we dont have the embedding:
adata=ppp.map_and_filter_based_on_embedding(adata)

# This calls DEGs and filters out treatments with to little cells
adata=ppp.postprocess_anndata(adata, n_top_genes=2000)


/home/manu/chemCPA/chemCPA/Preprocessing/post_preprocess.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['treatment']=[Specific_dict[a] for a in adata.obs['treatment']]


Treatments kept: 15860
Treatments removed: 5444


/home/manu/.local/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:216: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


A375
HA1E


/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/si

HELA


/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/manu/.local/lib/python3.10/si

AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
# It writes/reads faster when not using compression
adata.write(f'{save_path}/L1000_pp.h5ad', compression='gzip')

In [8]:
load_path='/home/manu/chemCPA/chemCPA/anndatas'
adata=sc.read(f'{load_path}/Norman_pp.h5ad')

In [10]:
adata.obs

,cell_line,treatment,treatment_dose_uM,treatment_time_h,treatment_type,dataset,control,pert_category,split
index,,,,,,,,,
AAACCTGAGAAGAAGC-1,K562,control,NaN,168.0,gene_knockout,Norman,1,K562_control,train
AAACCTGAGGCATGTG-1,K562,TSC22D1,NaN,168.0,gene_knockout,Norman,0,K562_TSC22D1,train
AAACCTGAGGCCCTTG-1,K562,EKLF+MAP2K6,NaN,168.0,gene_knockout,Norman,0,K562_EKLF+MAP2K6,train
AAACCTGCACGAAGCA-1,K562,control,NaN,168.0,gene_knockout,Norman,1,K562_control,train
AAACCTGCAGACGTAG-1,K562,CEBPE+RUNX1,NaN,168.0,gene_knockout,Norman,0,K562_CEBPE+RUNX1,ood
...,...,...,...,...,...,...,...,...,...
TTTGTCATCAGTACGT-8,K562,FOXA3,NaN,168.0,gene_knockout,Norman,0,K562_FOXA3,train
TTTGTCATCCACTCCA-8,K562,CELF2,NaN,168.0,gene_knockout,Norman,0,K562_CELF2,train
TTTGTCATCCCAACGG-8,K562,BCORL1,NaN,168.0,gene_knockout,Norman,0,K562_BCORL1,train
